## Выгрузка курса валют из Центробанка РФ

In [ ]:
import requests
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup

In [ ]:
start = dt.datetime(2022,5,9,0,0,0) # дата начала выгрузки
end = dt.datetime(2022,5,15,23,59,59) # дата конца

In [ ]:
def dauly_currency_cbr(my_date):
    
    my_date_url = my_date.strftime("%d/%m/%Y") # изменим тип даты на строку
    
    url = f"http://www.cbr.ru/scripts/XML_daily.asp?date_req={my_date_url}"
    html = requests.get(url)
    soup = BeautifulSoup(html.content)

    name = soup.findAll('name')
    value = soup.findAll('value')
    nominal = soup.findAll('nominal')
    charcode = soup.findAll('charcode')

    name_array = []
    value_array = []
    nominal_array = []
    charcode_array = []

    for i in range(0, len(name) - 1):
        if not charcode[i].contents:
            continue
        name_array.append(name[i].contents[0])
        value_array.append(value[i].contents[0])
        nominal_array.append(nominal[i].contents[0])
        charcode_array.append(charcode[i].contents[0])

    Df_Currency_cbr = pd.DataFrame({'name': name_array, 'value':value_array, 
                                    'nominal':nominal_array, 'charcode':charcode_array})
    
    Df_Currency_cbr['value'] = [x.replace(',', '.') for x in Df_Currency_cbr['value']]
    Df_Currency_cbr['value'] = Df_Currency_cbr['value'].astype(float)
    Df_Currency_cbr['nominal'] = Df_Currency_cbr['nominal'].astype(float)
    
    Df_Currency_cbr['value'] = Df_Currency_cbr['value'] / Df_Currency_cbr['nominal']
    
    Df_Currency_cbr['date_payment'] = my_date
    
    USD = Df_Currency_cbr.query('charcode == "USD"').iloc[0][1]
    
    Df_Currency_cbr['value'] = Df_Currency_cbr['value']  / USD
    #Df_Currency_cbr = Df_Currency_cbr.drop(['nominal','name'], axis=1)
    
    return Df_Currency_cbr

In [ ]:
df_dauly_currency = pd.DataFrame(columns=['name', 'value', 'nominal', 'charcode', 'date_payment'])

res = pd.date_range(start, end)

for date in res:
    df_day = dauly_currency_cbr(date)
    df_dauly_currency = df_dauly_currency.append(df_day)

df_dauly_currency = df_dauly_currency.drop(['name','nominal'], axis=1)

df_dauly_currency